In [1]:
#Importing
import os
import numpy as np
import nibabel as nib

### Thresolding the probability maps with following values
Thresholding probability maps with different threshold values. This was first performed using only 0.5, the other values was later added as all other models than the nnU-Net had to be thresholded again to see what value gave them the best score.
The code below is run for the LGA model, to threshold other probabiltiy maps the "path_to_val" variable has to be changed as well as the "probability_map" variable.

! The code below is based on terminal commands and should only have to be run once !

In [ ]:
#Getting list of the names of all the folder containing a probability map
all_files_list = [] #List that contains the Icometrix number of all subjects in the validation set
path_to_val = "/home/maarhus/Documents/Master/LST_data/LGA/MS_DATA_VAL" #Path to validation set

for path, dirs, files in os.walk(path_to_val): 
    if len(dirs) == 2: #Set to 2 because the LGA makes two folder
        all_files_list.append(os.path.basename(path))

In [4]:
#Thresholding
thr_list = [0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
probability_map = "ples_lga_0.2_rmFLAIR.nii"

for file in all_files_list:
    for num in thr_list:
        !cd "{path_to_val}/{file}" && fslmaths {probability_map} -thr {num} -bin "thr_{num}_map.nii"


In [5]:
#Unzipping folder
for file in all_files_list:
    for num in thr_list:
        !cd "{path_to_val}/{file}" && gunzip "thr_{num}_map.nii.gz"

In [6]:
#Make folder to contain all thresholded probability maps
for file in all_files_list:
    !cd "{path_to_val}/{file}" && mkdir thresholds

In [7]:
#Moving thresholded probability maps into its own folder
for file in all_files_list:
    for num in thr_list:
        !cd "{path_to_val}/{file}" && mv "thr_{num}_map.nii" thresholds

### Calculating dice score of all thresholds
This was first performed after the probability maps were thresholded at 0.5 and was later changed to check the dice score of multiple thresholds

In [17]:
#The function below use images in the form of numpy arrays to calcualte the scores
def my_dice(img1,img2):
 intersection = np.logical_and(img1, img2) #Finding intersection between the images
 union = np.logical_or(img1, img2) #Finding union between images
 dice = (2*np.sum(intersection))/(np.sum(union)+np.sum(intersection)) #Calculating dice score
 return dice

In [11]:
#Saving a list of all the best threshold found
all_best_threshold = []

#Finding the dice scores for every thresholded probability map for all subjects in validation set
for file in all_files_list:
    #Starting with a score of 0
    best_score = 0 
    
    #For all thresholds in list
    for num in thr_list:
        #Loading thresholded probability map, NIfTY image
        path_pred = f'{path_to_val}/{file}/thresholds/thr_{num}_map.nii'

        #Loading true lesion mask, NIfTY image 
        path_mask = f'{path_to_val}/{file}/lesions_binarised.nii'

        #Making the NIfTY images into numpy arrays to calculate score
        pred_image = nib.load(path_pred).get_fdata()
        mask_image = nib.load(path_mask).get_fdata()

        #Calculating dice score and printing the name of the folder, threshold value and corresponding score
        dice_score = my_dice(pred_image, mask_image)
        print(f'Score of prediction in folder {file} with threshold {num} is: {dice_score}')
        
        #If the score so far has been the best, set best_score equal to this
        if dice_score > best_score:
            best_score = dice_score
            best_thr = num

    #After iterating through all the thresholds, print which threshold that resulted in the largest dice score
    print(f'The largest dice score from folder {file} is {best_score} with thr = {best_thr}')

    #Adding largest threshols from current folder to list of all best thresholds
    all_best_threshold.append(best_thr)

Score of prediction in folder 2492322305 with threshold 0.05 is: 0.6916266126992158
Score of prediction in folder 2492322305 with threshold 0.1 is: 0.6949588477366255
Score of prediction in folder 2492322305 with threshold 0.15 is: 0.6917488323819408
Score of prediction in folder 2492322305 with threshold 0.2 is: 0.6924888772572625
Score of prediction in folder 2492322305 with threshold 0.3 is: 0.6903311258278145
Score of prediction in folder 2492322305 with threshold 0.4 is: 0.688550840672538
Score of prediction in folder 2492322305 with threshold 0.5 is: 0.6851453175457481
Score of prediction in folder 2492322305 with threshold 0.6 is: 0.6834551854860547
Score of prediction in folder 2492322305 with threshold 0.7 is: 0.6817192600652884
Score of prediction in folder 2492322305 with threshold 0.8 is: 0.6806883365200764
Score of prediction in folder 2492322305 with threshold 0.9 is: 0.6783465644675609
The largest dice score from folder 2492322305 is 0.6949588477366255 with thr = 0.1
Sco

In [21]:
#Finding what threshold value ended up giving the best dice score the most times
#This was used as the value to threshold the probability maps in the test set at
best_thr_overall = max(all_best_threshold, key=all_best_threshold.count)
print(best_thr_overall)

0.1


### Thresolding test files with value found from validation set 

In [2]:
test_files = []
path_to_test = "/home/maarhus/Documents/Master/LST_data/LGA/MS_DATA_TEST"
for path, dirs, files in os.walk(path_to_test):
    #I know none of my subdirectories will have their own subfolders 
    if len(dirs) == 2: #Set to 2 because the LGA makes two foldera
        test_files.append(os.path.basename(path))


In [23]:
for file in test_files:
    !cd "{path_to_test}/{file}" && fslmaths ples_lga_0.2_rmFLAIR.nii -thr {num} -bin "thr_{best_thr_overall}_map.nii"


In [24]:
for file in test_files:
    !cd "{path_to_test}/{file}" && gunzip "thr_{best_thr_overall}_map.nii.gz"